## Create images from signals.joblib

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import joblib
import time
import os
import random

### Helper code

In [2]:
from dataclasses import dataclass, field
import numpy as np
#from stingray.utils import fftshift, fft2, ifftshift, fft
from scipy.fft import fftshift, fft2, ifftshift, fft
from scipy.linalg import toeplitz
import matplotlib.pyplot as plt

@dataclass(repr=False)
class Bispectrum2D:
    '''
    Make Bispectrum dataclass from 1D signal with frequency in Hertz.
    
    Parameters:
    -----------
        signal: The 1-Dimensional signal in (n,) numpy.ndarray
        freqsample:  The sample frequency in Hertz.  
        window: 'None', 'hanning', 'triangular'
    
    Author: jkmackie
    
    References
    ----------
    Matteo Bachetti, et al., stingray v1.0 code, DOI: https://zenodo.org/record/6394742, 2022.
    
    '''    
    signal: np.ndarray
    freqsample: float
    window_name: str
    dt: float = field(init=False)
    n: int = field(init=False)  #number of data points in signal
    maxlag: int = field(init=False)
    lagindex: np.ndarray = field(init=False)
    cum3_dim: int = field(init=False)
            
    def __post_init__(self):
        self.dt = 1 / self.freqsample        
        self.n = self.signal.shape[0]
        self.maxlag = int(self.n/2)
        self.lagindex = np.arange(-self.maxlag, self.maxlag + 1)
        self.cum3_dim = 2 * self.maxlag + 1
        self._calc_bispectrum()
        
    def _calc_bispectrum(self):
        self._cumulant3()
        self._window()
        self._bispectrum()
    
    def _cumulant3(self):
        '''Biased cumulant estimate.'''
        self.cum3 = np.zeros((self.cum3_dim, self.cum3_dim))  #include zeros matrix to reset calc 
        ind = np.arange((self.n - self.maxlag)-1, self.n) #consecutive idx from (n-maxlag-1) to n
        ind_t = np.arange(self.maxlag, self.n)
        zero_maxlag = np.zeros((1, self.maxlag))
        zero_maxlag_t = zero_maxlag.T
        sig = np.reshape(self.signal, (1,len(self.signal)))  #Reshape original self.sig
        sig = sig - np.mean(sig)                       #sig is 1xn row vector of counts.
        rev_sig = np.array([sig[0][::-1]])
        col = np.concatenate((sig.T[ind], zero_maxlag_t), axis=0)
        row = np.concatenate((rev_sig[0][ind_t], zero_maxlag[0]), axis=0)
        toep = toeplitz(col, row)
        rev_sig_repeat = np.repeat(rev_sig, [2 * self.maxlag + 1], axis=0) #n repeats
        #toep is n x (n-1).  It must be square to be a circulant.
        self.cum3 = (self.cum3 + np.matmul(np.multiply(toep, rev_sig_repeat), toep.T)) / self.n        
                
    def _window(self):        
        n = np.arange(self.cum3_dim)         #Total wind data points matches cum3_dim        
        self.window = np.zeros(self.cum3_dim)  
        if self.window_name == 'None':
            return
        if self.window_name == 'hanning':
            hanning = 0.5 * (1 - np.cos(2 * np.pi * n / (self.cum3_dim-1)))
            wind2D = np.tile(hanning,(self.cum3_dim,1))  #Make 2D wind by repeating rows N times.
            self.window[:self.maxlag + 1] = hanning[self.maxlag:]        
        if self.window_name == 'triangular':
            N_div_2 = int((np.floor((self.cum3_dim - 1) / 2)))
            triangular = 1 - np.abs((n - (N_div_2)) / self.cum3_dim)            
            wind2D = np.tile(triangular,(self.cum3_dim,1))  #Make 2D wind by repeating rows N times.
            self.window[:maxlag + 1] = triangular[self.maxlag:]    
        self.window[self.maxlag:] = 0
        # Put wind in toeplitz.  Each row of final wind is sliding hanning.
        row = np.concatenate(([self.window[0]], np.zeros(2 * self.maxlag)))
        toep_matrix = toeplitz(self.window, row)
        toep_matrix += np.tril(toep_matrix, -1).transpose()
        self.window = toep_matrix[..., ::-1] * wind2D * wind2D.T
    
    def _bispectrum(self):
        if self.window_name == 'None':
            self.bispec = fftshift(fft2(ifftshift(self.cum3)))
        else:
            self.bispec = fftshift(fft2(ifftshift(self.cum3*self.window)))         
        self.freqvals = 0.5 * self.freqsample * self.lagindex / self.maxlag        
        self.bispec_mag = np.abs(self.bispec)
        self.bispec_phase = np.angle(self.bispec)
        
    def plot_cum3(self):
        lags = self.lagindex * self.dt
        fig, ax1 = plt.subplots(1,1,figsize=(6,6)) #gist has (11,11)
        contplot1 = ax1.contourf(lags, lags, self.cum3, levels=100, cmap=plt.cm.Spectral_r)
        ax1.set_title('Third Order Cumulant'); ax1.set_xlabel('lag 1 values'); ax1.set_ylabel('lags 2 values');
    
    def plot_bispec_magnitude(self):    
        fig, ax1 = plt.subplots(1,1,figsize=(6,6))
        contplot1 = ax1.contourf(self.freqvals, self.freqvals, self.bispec_mag, levels=100, cmap=plt.cm.Spectral_r)
        ax1.set_title('Signal Bispectrum Magnitude'); ax1.set_xlabel('freq 1 Hz'); ax1.set_ylabel('freq 2 Hz');
        #plt.colorbar(contplot1)

#bs2D = Bispectrum2D(signal=y, freqsample=16_000, window_name='hanning')

In [3]:
def save_bispec_im_to_temp():
    # Save bispectrum images

    #Params
    colnames = ['dc1','nc1']
    segment_points = 2**12-1
    start_indices = range(0, 800_000 - (segment_points) + 1, segment_points)
    #start_indices = range(0, 9000, segment_points)  #create only six images

    for colname in colnames:
        for start_index in start_indices:
            end_index = start_index + segment_points
            y = df[colname].iloc[start_index : end_index].copy().values
            bs2D = Bispectrum2D(signal=y, freqsample=16_000, window_name='hanning')

            fig, ax1 = plt.subplots(1,1,figsize=(11,11))
            ax1.axis('off')
            contplot = ax1.contourf(bs2D.freqvals, bs2D.freqvals, bs2D.bispec_mag, levels=100, cmap=plt.cm.Spectral_r)
            plt.savefig(f'./temp/{colname}_hanning_{start_index}-{end_index}.png')    
            print('file saved:', f'{colname}_hanning_{start_index}-{end_index}.png', '| min elapsed:', round(((time.time()-start)/60),2))
            plt.close()
            #plt.show()

In [4]:
def crop_imag(imageFilePath: str, shakerange=20, shake_on=None):
    '''Crop 1100x1100 images to 224X224.  Optionally shake from center-crop
    location (438,438,662,662).  Return 224X224 image.'''
    from PIL import Image
    im = Image.open(imageFilePath)
    rix = random.randint(-shakerange, shakerange) #get random int for x1, x2
    riy = random.randint(-shakerange, shakerange) #get random int for y1, y2
    if shake_on == True:
        shaken_centercrop = (438+rix, 438+riy, 662+rix, 662+riy) 
        imCropped = im.crop(shaken_centercrop) #im.crop(x1, y1, x2, y2) - (x1,y1) top-left coord; (x2,y2) bottom-right coord
    else:
        centercrop = (438, 438, 662, 662)
        imCropped = im.crop(centercrop)    
    return imCropped

def preprocess_3d_image(imPath: str):
    '''Preprocess PIL format image for VGG16.'''
    from tensorflow.keras.preprocessing import image
    from tensorflow.keras.applications.vgg16 import preprocess_input    
    img = image.load_img(imPath)         #load image into PIL format
    arr = image.img_to_array(img)        #convert to np.array
    arr = preprocess_input(arr)          #covert to BGR and zero-center color channels for VGG16
    return image.array_to_img(arr)       #return processed img

In [5]:
def preprocess_and_save(image_folder, save_folder):
    '''Crop images to 224X224.  Do VGG16-specific preprocessing.
    Save processed images to save_folder.'''
    # Reference:  https://keras.io/api/applications/#usage-examples-for-image-classification-models
    for im_name in os.listdir(img_fold):
        fp = img_fold + im_name
        im_cropped=crop_imag(imageFilePath=fp, shakerange=10, shake_on=True)
        im_cropped.save(img_fold+im_name)         #Saving after crop and before preprocessing may be required!
        im_procd = preprocess_3d_image(imPath=fp)
        #display(im_procd)
        im_procd.save(save_fold+im_name)

### Load data

In [6]:
df = joblib.load('signals.joblib')
print('number of nulls in df:', df.isnull().sum().sum())
df.describe().round(3)

number of nulls in df: 0


,nc1,lc1,dc1
count,800001.000,800001.000,800001.000
mean,0.000,-0.000,-0.000
std,0.611,0.646,0.776
min,-2.834,-5.926,-29.329
25%,-0.478,-0.467,-0.493
50%,0.070,0.063,0.030
75%,0.515,0.496,0.504
max,2.687,6.178,26.145


In [7]:
df.tail(3)

,nc1,lc1,dc1
799998,0.653171,-0.637368,0.240326
799999,0.047256,0.204431,0.319485
800000,-0.682212,0.365421,0.589967


In [8]:
start = time.time()

### Save images to disk.  Preprocess images.

In [9]:
#Create bispectrum images.  Save to ./temp folder.  Folder must exist.
save_bispec_im_to_temp()

file saved: dc1_hanning_0-4095.png | min elapsed: 0.18
file saved: dc1_hanning_4095-8190.png | min elapsed: 0.36
file saved: dc1_hanning_8190-12285.png | min elapsed: 0.55
file saved: nc1_hanning_0-4095.png | min elapsed: 0.72
file saved: nc1_hanning_4095-8190.png | min elapsed: 0.89
file saved: nc1_hanning_8190-12285.png | min elapsed: 1.06


In [10]:
#Preprocess and save images.  Folders must exist.
img_fold = './temp/'
save_fold = './preproc/'

preprocess_and_save(image_folder=img_fold, save_folder=save_fold)

In [11]:
end = time.time()
print('minutes elapsed:', round((end-start)/60,2))

minutes elapsed: 1.08
